In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import numpy as np
import copy
import re

In [ ]:
df = pd.read_csv(
    "./survey_preprocessed.csv",
    sep=";",
    quoting=csv.QUOTE_MINIMAL,
    encoding="utf-8",
    engine="python"
)

,commento,valutazione,tossicita,age_group,gender,utente_id,comment_cleaned,emoji_set,emoji_count,parole_maiusc,...,SMOGIndex,RIX,DaleChallIndex,comment_tokenized,comment_tok=2,comment_tok=3,comment_lemmatized,VAD_valence,VAD_arousal,VAD_dominance
0,is this a satire subreddit?,Non toxic,0,1.0,1,1,satire subreddit,set(),0,[],...,8.477226,1.0,15.889200,"['satire', 'subreddit']","[('satire', 'subreddit')]",[],"['satire', 'subreddit']",-0.084000,0.310000,-0.056000
1,Dayum.,Non toxic,0,1.0,1,1,dayum,set(),0,[],...,3.000000,0.0,15.839600,['dayum'],[],[],['dayum'],0.000000,0.000000,0.000000
2,And they wouldn’t be scrubbing the floors if t...,Non toxic,0,1.0,1,1,would scrubbing floors employed,set(),0,[],...,3.000000,2.0,12.040900,"['would', 'scrubbing', 'floors', 'employed']","[('would', 'scrubbing'), ('scrubbing', 'floors...","[('would', 'scrubbing', 'floors'), ('scrubbing...","['would', 'scrub', 'floor', 'employ']",-0.102000,0.282500,-0.358500
3,Good point. I mean does Nebraska really exist?...,Non toxic,0,1.0,1,1,good point mean nebraska really exist ever met...,set(),0,[],...,10.745967,3.0,5.233000,"['good', 'point', 'mean', 'nebraska', 'really'...","[('good', 'point'), ('point', 'mean'), ('mean'...","[('good', 'point', 'mean'), ('point', 'mean', ...","['good', 'point', 'mean', 'nebraska', 'really'...",0.326000,-0.267333,0.086333
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,Non toxic,0,1.0,1,1,yuumi nami lulu zilean enhancer plays team,set(),0,[],...,8.477226,1.0,13.881486,"['yuumi', 'nami', 'lulu', 'zilean', 'enhancer'...","[('yuumi', 'nami'), ('nami', 'lulu'), ('lulu',...","[('yuumi', 'nami', 'lulu'), ('nami', 'lulu', '...","['yuumi', 'nami', 'lulu', 'zilean', 'enhancer'...",0.687000,0.294500,0.521500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270,Commies are going to commie. These people hate...,Extremely violent,3,1.0,0,211,commies going commie people hate want die,set(),0,[],...,3.000000,1.0,4.858629,"['commies', 'going', 'commie', 'people', 'hate...","[('commies', 'going'), ('going', 'commie'), ('...","[('commies', 'going', 'commie'), ('going', 'co...","['commie', 'go', 'commie', 'people', 'hate', '...",-0.329714,0.375857,0.045000
5271,Yeah but then the good guys win and rape you t...,Extremely violent,3,1.0,0,211,yeah good guys win rape death occupy nation in...,set(),0,[],...,10.745967,1.0,10.973067,"['yeah', 'good', 'guys', 'win', 'rape', 'death...","[('yeah', 'good'), ('good', 'guys'), ('guys', ...","[('yeah', 'good', 'guys'), ('good', 'guys', 'w...","['yeah', 'good', 'guy', 'win', 'rape', 'death'...",0.012750,0.015250,0.121125
5272,If you're white and use your gun to defend you...,A little violent,1,1.0,0,211,white use gun defend arrested racist get life ...,set(),0,[],...,8.477226,1.0,3.955289,"['white', 'use', 'gun', 'defend', 'arrested', ...","[('white', 'use'), ('use', 'gun'), ('gun', 'de...","[('white', 'use', 'gun'), ('use', 'gun', 'defe...","['white', 'use', 'gun', 'defend', 'arrest', 'r...",-0.115429,0.365714,0.212857
5273,I will rape Elon Musk,A little violent,1,1.0,0,211,rape elon musk,set(),0,[],...,3.000000,0.0,15.938800,"['rape', 'elon', 'musk']","[('rape', 'elon'), ('elon', 'musk')]","[('rape', 'elon', 'musk')]","['rape', 'elon', 'musk']",-0.469000,0.310000,-0.020000


In [42]:
df.columns

Index(['commento', 'valutazione', 'tossicita', 'age_group', 'gender',
       'utente_id', 'comment_cleaned', 'emoji_set', 'emoji_count',
       'parole_maiusc', 'pos_tag', 'nrc_affect_dict', 'nrc_raw_scores',
       'nrc_top_emotions', 'nrc_affect_frequencies', 'comm_polarita',
       'Kincaid', 'ARI', 'Coleman-Liau', 'FleschReadingEase',
       'GunningFogIndex', 'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex',
       'comment_tokenized', 'comment_tok=2', 'comment_tok=3',
       'comment_lemmatized', 'VAD_valence', 'VAD_arousal', 'VAD_dominance'],
      dtype='object')

### CONTEGGIO PAROLE PIÙ FREQUENTI

In [ ]:
from nltk import FreqDist

all_tokens = [token for list in df['comment_tokenized'] for token in list]
freq_dist = FreqDist(all_tokens)

freq_dist.most_common(20)

In [ ]:
freq_dist.plot(20)

### DISPERSIONE DELLE PAROLE NEL TESTO

In [ ]:
nltk_text = nltk.Text(all_tokens)

# grafico che mostra la dispersione di una parola all'interno del testo (inizio, fne, centrale)
nltk_text.dispersion_plot(['people','fuck','go', 'shit','like', 'fucking', 'would'])

### TF-IDF (parole più importanti)

In [ ]:
df["comment_uniti"] = df["comment_tokenized"].apply(lambda tokens: " ".join(tokens))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer1 = TfidfVectorizer()
X = vectorizer1.fit_transform(df["comment_uniti"])
feature_names1 = vectorizer1.get_feature_names_out()
tfidf_df1 = pd.DataFrame(X.toarray(), columns=vectorizer1.get_feature_names_out())
tfidf_df1

In [ ]:
word_scores = np.asarray(X.sum(axis=0)).ravel()
sorted_indices = word_scores.argsort()[::-1]

N = 20
top_words = [(feature_names1[i], word_scores[i]) for i in sorted_indices[:N]]
print("Top parole globali:", top_words)

words, scores = zip(*top_words)
plt.figure(figsize=(10, 6))
plt.barh(words, scores)
plt.xlabel("TF-IDF Score (sommato su tutti i documenti)")
plt.title(f"Top {N} parole nel corpus")
plt.gca().invert_yaxis()
plt.show()

considero anche bigrammi o trigrammi, consideri anche sequenze di 2 o 3 parole consecutive come feature, per vedere se ci sono delle parole che singolarmente non hanno significato, ma insieme lo acquisiscono (es. New York)


In [ ]:
vectorizer2 = TfidfVectorizer(ngram_range=(2,3)) 
X = vectorizer2.fit_transform(df["comment_uniti"])
feature_names2 = vectorizer2.get_feature_names_out()
tfidf_df2 = pd.DataFrame(X.toarray(), columns=vectorizer2.get_feature_names_out())
tfidf_df2

In [ ]:
word_scores = np.asarray(X.sum(axis=0)).ravel()
sorted_indices = word_scores.argsort()[::-1]

N = 20
top_words = [(feature_names2[i], word_scores[i]) for i in sorted_indices[:N]]
print("Top parole globali:", top_words)

words, scores = zip(*top_words)
plt.figure(figsize=(10, 6))
plt.barh(words, scores)
plt.xlabel("TF-IDF Score (sommato su tutti i documenti)")
plt.title(f"Top {N} parole nel corpus")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df.drop(columns=['comment_uniti'], inplace=True)
df